In [1]:
import os

In [2]:
%pwd

'a:\\mlflow\\Heart-Failure-Prediction-with-Machine-Learning-MLflow\\research'

In [3]:
os.listdir(os.getcwd())

['1.data_ingestion.ipynb',
 '2.data_validation.ipynb',
 '3.data_transformation.ipynb',
 '4.model_trainer.ipynb',
 '5.model_evaluation.ipynb',
 'trials.ipynb']

In [4]:
os.chdir("../")

In [5]:
%pwd

'a:\\mlflow\\Heart-Failure-Prediction-with-Machine-Learning-MLflow'

In [6]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/Rutvik46/Heart-Failure-Prediction-with-Machine-Learning-MLflow.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="Rutvik46" 
os.environ["MLFLOW_TRACKING_PASSWORD"]="fdf9d63cdff88d1ed6fd22bc2f3d34a8c0672d1a"

In [24]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir:str
    test_data_path:Path
    model_path:Path
    all_params:dict
    metrics_file_path:Path
    target_column:str
    mlflow_url:str

In [8]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [25]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH,schema_filepath=SCHEMA_FILE_PATH):
        print(f"Config file path: {config_filepath}")
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.model_evaluation.root_dir])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        param = self.params.Support_Vector_Machine
        schema = self.schema.TARGET_COLUMN

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=Path(config.test_data_path),
            model_path=Path(config.model_path),
            all_params=param,
            metrics_file_path=Path(config.metrics_file_path),
            target_column=schema.name,
            mlflow_url="https://dagshub.com/Rutvik46/Heart-Failure-Prediction-with-Machine-Learning-MLflow.mlflow"
        )

        return model_evaluation_config

In [26]:
import pandas as pd
from mlProject import logger
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, precision_score, recall_score
from urllib.parse import urlparse
from sklearn.preprocessing import StandardScaler
import mlflow
import mlflow.sklearn
import numpy as np
import joblib
from mlProject.utils.common import save_json 


In [ ]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def evaluation_metrics(self,actual,pred):
        Accuracy = accuracy_score(actual,pred)
        F1_Score = f1_score(actual,pred)
        Precision_Score = precision_score(actual,pred)
        Recall_Score = recall_score(actual,pred)
        AUC_Score = roc_auc_score(actual, pred)
        
        return Accuracy, F1_Score, Precision_Score, Recall_Score, AUC_Score

    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        scaler = StandardScaler()

        X_test = test_data.drop([self.config.target_column], axis=1)
        X_test = scaler.fit_transform(X_test)

        Y_test = test_data[self.config.target_column]


        mlflow.set_registry_uri(self.config.mlflow_url)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_qualities = model.predict(X_test)
            #print(predicted_qualities)

            (Accuracy, F1_Score, Precision_Score, Recall_Score, AUC_Score) = self.evaluation_metrics(Y_test, predicted_qualities)
            
            # Saving metrics as local
            scores = {"Accuracy":Accuracy, "F1_Score":F1_Score, "Precision_Score":Precision_Score, "Recall_Score":Recall_Score, "AUC_Score":AUC_Score}
            save_json(path=Path(self.config.metrics_file_path), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("Accuracy",Accuracy)
            mlflow.log_metric("F1_Score",F1_Score)
            mlflow.log_metric("Precision_Score",Precision_Score)
            mlflow.log_metric("Recall_Score",Recall_Score)
            mlflow.log_metric("AUC_Score",AUC_Score)


            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="Support Vector Machine")
            else:
                mlflow.sklearn.log_model(model, "model")



In [28]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()

except Exception as e:
    logger.exception(e)
    raise e

Config file path: config\config.yaml
[2025-07-21 21:56:26,554 : INFO: yaml file: config\config.yaml loaded successfully]
[2025-07-21 21:56:26,557 : INFO: yaml file: params.yaml loaded successfully]
[2025-07-21 21:56:26,558 : INFO: yaml file: schema.yaml loaded successfully]
[2025-07-21 21:56:26,562 : INFO: created directory at: artifacts/model_evaluation]
[2025-07-21 21:56:26,611 : WARNING: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /Rutvik46/Heart-Failure-Prediction-with-Machine-Learning-MLflow.mlflow/api/2.0/mlflow/runs/create]
[0 1 1 1 0 1 1 0 1 1 1 0 0 0 1 1 0 1 1 1 1 1 0 1 1 1 0 1 0 0 1 0 1 1 1 1 1
 0 1 1 1 1 1 0 0 1 1 1 1 1 0 0 1 1 0 0 0 1 1 1 0 0 0 1 0 1 1 1 1 1 1 0 1 1
 0 1 0 1 0 1 0 1 1 0 1 1 0 1 0 0 0 1 1 0 1 0 1 0 1 1 0 1 1 0 1 1 0 0 1 0 1
 1 0 0 1 0 1 0 1 1 1 1 1 1 0 0 0 0 0 0 0 1 1 0 1 0 1 1 0 1 0 0 1 1 1 1 1 0
 0 0 0 1 1 0 1 0 0 0 0 1 0 0 1 0 1 1 1 1 0

Registered model 'Support Vector Machine' already exists. Creating a new version of this model...
2025/07/21 21:56:33 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: Support Vector Machine, version 4
Created version '4' of model 'Support Vector Machine'.
